In [1]:
import pandas as pd
import numpy as np
import math
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from scipy import stats
from collections import Counter
from tqdm import tqdm  # Import tqdm for progress tracking

# --- CONFIGURATION BASED ON TEXT ---
# Section 3.2.2: Low-Complexity Region Quantification
SEG_WINDOW = 12
SEG_TRIGGER = 2.2
SEG_EXTENSION = 2.5
LOW_COMPLEXITY_MAX_FRACTION = 0.20  # >20% exclusion threshold

# Section 3.2.2: Physicochemical Validation
HALO_MEDIAN_PI_MIN = 4.2
HALO_MEDIAN_PI_MAX = 4.8
KS_ALPHA = 0.01  # Significance level

# Amino Acid Composition thresholds (Halobacteria)
HALO_DE_PCT_MIN = 20.0     # Asp+Glu > 20%
HALO_K_PCT_MAX = 2.0       # Lys < 2%
HALO_DE_K_RATIO_MIN = 10.0 # (Asp+Glu)/Lys > 10

# --- HELPER FUNCTIONS ---

def calculate_shannon_entropy(sequence):
    """Calculates Shannon entropy in bits for a sequence string."""
    L = len(sequence)
    if L == 0: return 0
    counts = Counter(sequence)
    entropy = 0
    for count in counts.values():
        p = count / L
        entropy -= p * math.log2(p)
    return entropy

def seg_low_complexity_filter(sequence, window=12, trigger=2.2, extension=2.5):
    """
    Implements the SEG algorithm logic:
    1. Identification: Sliding window entropy < trigger.
    2. Extension: Merges overlapping/adjacent windows if entropy remains low (< extension).
    Returns the fraction of the sequence covered by low-complexity regions.
    """
    N = len(sequence)
    if N < window:
        return 0.0
    
    # 1. Identify "Raw" Low Complexity Segments
    is_lc = np.zeros(N, dtype=bool)
    
    # We first find all windows that trigger the low complexity state
    trigger_indices = []
    for i in range(N - window + 1):
        subseq = sequence[i : i+window]
        H = calculate_shannon_entropy(subseq)
        if H < trigger:
            trigger_indices.append(i)
            # Mark the window itself
            is_lc[i : i+window] = True
            
    # 2. Extension Phase (Simplistic implementation of SEG extension)
    # If a window is "triggered", we check if we can extend it left/right 
    # while keeping entropy below the 'extension' threshold.
    # Note: True SEG optimization is complex; this approximates the 'extension' param
    # by allowing the merge of regions if the combined entropy is low.
    
    # A simplified interpretation for pipeline purposes:
    # If we are in a low complexity state, we are more tolerant (extension threshold)
    # regarding what we consider "still low complexity" nearby.
    
    # For this pipeline, we will refine the mask:
    # Any block marked 'True' is a seed. We check adjacent windows against 'extension'.
    # (Since standard SEG code is C/C++, this python approximation ensures we catch the regions).
    
    # Re-scan to extend seeds using the higher 'extension' entropy threshold
    final_lc_mask = is_lc.copy()
    
    # Identify continuous blocks
    # (In a full SEG implementation, this involves iterative extension. 
    # Here, we ensure that if we are "near" a trigger, we use the extension threshold).
    
    # Refined pass: Check all windows again against EXTENSION threshold
    # But only apply if they overlap/touch an existing TRIGGERED region.
    
    # Actually, a safer Pythonic approximation for "Extension" in this context
    # is often interpreted as: "Once triggered, the condition to STAY in low-complexity
    # is the extension value."
    
    # Let's perform a second pass to bridge gaps using the extension threshold
    for i in range(N - window + 1):
        if is_lc[i:i+window].any(): # If it touches a triggered region
            subseq = sequence[i : i+window]
            H = calculate_shannon_entropy(subseq)
            if H < extension: # Higher threshold (easier to pass)
                final_lc_mask[i : i+window] = True

    return np.sum(final_lc_mask) / N

def get_biophysical_stats(sequence):
    """Returns pI and Amino Acid percentages."""
    try:
        # Standardize
        clean_seq = sequence.upper().replace('X', '').replace('B', '').replace('Z', '').replace('J', '')
        if not clean_seq: return None
        
        X = ProteinAnalysis(clean_seq)
        pI = X.isoelectric_point()
        
        # AA Composition
        aa_counts = X.count_amino_acids()
        total_len = len(clean_seq)
        if total_len == 0: return None
        
        D = aa_counts.get('D', 0)
        E = aa_counts.get('E', 0)
        K = aa_counts.get('K', 0)
        
        pct_DE = ((D + E) / total_len) * 100.0
        pct_K = (K / total_len) * 100.0
        
        ratio_DE_K = (D + E) / K if K > 0 else (D + E) # Handle divide by zero
        
        return pI, pct_DE, pct_K, ratio_DE_K
    except Exception as e:
        return None

# --- MAIN PIPELINE ---

def validation_pipeline(input_csv, output_data, output_metadata):
    print(f"--- Loading Dataset: {input_csv} ---")
    df = pd.read_csv(input_csv)
    
    # Enable tqdm for pandas apply operations
    tqdm.pandas()
    
    # --- 1. Low-Complexity Region Quantification ---
    print(f"Stage 1: Quantifying Low-Complexity Regions (SEG: W={SEG_WINDOW}, T={SEG_TRIGGER}, E={SEG_EXTENSION})...")
    
    # Apply the SEG filter logic with progress bar
    df['lc_fraction'] = df['Sequence'].progress_apply(lambda x: seg_low_complexity_filter(x, SEG_WINDOW, SEG_TRIGGER, SEG_EXTENSION))
    
    # Flag/Exclude > 20%
    initial_count = len(df)
    df_clean = df[df['lc_fraction'] <= LOW_COMPLEXITY_MAX_FRACTION].copy()
    dropped_count = initial_count - len(df_clean)
    if dropped_count > 0:
        print(f"  -> Flagged/Excluded {dropped_count} sequences with >20% low-complexity content.")
    
    # --- 2. Calculate Biophysical Properties ---
    print("Stage 2: Calculating Biophysical Properties...")
    # This returns a dataframe of stats with progress bar
    stats_data = df_clean['Sequence'].progress_apply(lambda x: pd.Series(get_biophysical_stats(x)))
    stats_data.columns = ['pI', 'pct_DE', 'pct_K', 'ratio_DE_K']
    
    # Merge back
    df_clean = pd.concat([df_clean, stats_data], axis=1)
    df_clean = df_clean.dropna(subset=['pI']) # Drop failures

    # --- 3. Grouping and Physicochemical Validation ---
    print("Stage 3: Grouping by Organism Id and Validating...")
    grouped = df_clean.groupby(['Organism Id', 'Proteome Id'])
    
    valid_organism_ids = []
    metadata_records = []

    # Wrap grouped iterator with tqdm for progress bar
    for (org_id, prot_id), group in tqdm(grouped, total=len(grouped), desc="Validating Organisms"):
        pi_values = group['pI'].values
        
        # --- A. Isoelectric Point Distribution ---
        # 1. Median Check (Strict ~4.2 - 4.8)
        median_pi = np.median(pi_values)
        is_median_valid = (HALO_MEDIAN_PI_MIN <= median_pi <= HALO_MEDIAN_PI_MAX)
        
        # 2. KS Test
        # Text: "Distribution comparisons via Kolmogorov–Smirnov test (alpha=0.01) will verify significant shifts."
        # We compare against the Halophilic reference (Normal dist, mean=4.5, std=1.0)
        # Note on KS Test interpretation:
        # Null Hypothesis (H0): The sample comes from the reference distribution.
        # If p-value < alpha, we REJECT H0 (It is NOT the reference distribution).
        # Therefore, we WANT p-value > alpha (we want it to look like the reference).
        ks_stat, ks_pvalue = stats.kstest(pi_values, 'norm', args=(4.5, 1.0))
        
        # If p > 0.01, we cannot reject that it is Halophilic (Pass).
        # If p < 0.01, it is significantly different from Halophilic profile (Fail).
        is_ks_valid = ks_pvalue > KS_ALPHA
        
        # --- B. Amino Acid Composition Profiles ---
        # "Asp+Glu > 20%, Lys < 2%, Ratio > 10"
        avg_pct_DE = group['pct_DE'].mean()
        avg_pct_K = group['pct_K'].mean()
        avg_ratio = group['ratio_DE_K'].mean()
        
        is_comp_valid = (
            (avg_pct_DE >= HALO_DE_PCT_MIN) and
            (avg_pct_K <= HALO_K_PCT_MAX) and
            (avg_ratio >= HALO_DE_K_RATIO_MIN)
        )

        # --- FINAL DECISION ---
        # The organism must pass the Median pI check AND the KS distribution check.
        # Composition checks reinforce the signature.
        
        # Strict adherence to "verify significant shifts":
        # We enforce Median pI range and KS distribution fit.
        final_valid = is_median_valid and is_ks_valid and is_comp_valid

        if final_valid:
            valid_organism_ids.append(org_id)
            
        metadata_records.append({
            'Proteome Id': prot_id,
            'Organism Id': org_id,
            'Sequence Count': len(group),
            'Median pI': round(median_pi, 4),
            'KS Statistic': round(ks_stat, 4),
            'KS P-Value': round(ks_pvalue, 6),
            'Avg Asp+Glu%': round(avg_pct_DE, 2),
            'Avg Lys%': round(avg_pct_K, 2),
            'DE/K Ratio': round(avg_ratio, 2),
            'Validation Status': 'Passed' if final_valid else 'Failed'
        })

    # --- 4. Save Results ---
    # Filter Original Data
    final_df = df_clean[df_clean['Organism Id'].isin(valid_organism_ids)].copy()
    output_cols = ['Proteome Id', 'Organism Id', 'Entry', 'Sequence']
    final_df = final_df[output_cols]
    
    final_df.to_csv(output_data, index=False)
    
    # Save Metadata
    meta_df = pd.DataFrame(metadata_records)
    meta_df.to_csv(output_metadata, index=False)
    
    print(f"--- Processing Complete ---")
    print(f"Organisms Passed: {len(valid_organism_ids)} out of {len(grouped)}")
    print(f"Validated Data: {output_data}")
    print(f"Metadata: {output_metadata}")

if __name__ == "__main__":
    validation_pipeline(
        'thermoproteota_data.csv', 
        'thermoproteota_validated_data.csv', 
        'thermoproteota_validated_metadata.csv'
    )

--- Loading Dataset: thermoproteota_data.csv ---
Stage 1: Quantifying Low-Complexity Regions (SEG: W=12, T=2.2, E=2.5)...


100%|██████████| 229906/229906 [02:37<00:00, 1459.55it/s]


  -> Flagged/Excluded 29109 sequences with >20% low-complexity content.
Stage 2: Calculating Biophysical Properties...


100%|██████████| 200797/200797 [00:12<00:00, 15693.27it/s]


Stage 3: Grouping by Organism Id and Validating...


Validating Organisms: 100%|██████████| 98/98 [00:00<00:00, 1315.49it/s]

--- Processing Complete ---
Organisms Passed: 0 out of 98
Validated Data: thermoproteota_validated_data.csv
Metadata: thermoproteota_validated_metadata.csv
